In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [31]:
# Lstm takes continues inputs and generates similar outputs
input_dim = 3; hidden_dim= 128;
lstm = nn.LSTM(input_dim, hidden_dim)

In [32]:
T = 5 # time sequence (length of your input)
B = 1 # minibatching (batchsize)
inputs=  [torch.randn(1, 3) for _ in range(T)]

In [33]:
inputs # this is a training instance
inputs[0].view(1,1,-1).shape

torch.Size([1, 1, 3])

In [34]:
outputs1 = [];
# hiddens1 contains tuple (hidden, cell) and the hidden and outputs[i]
# is the same thing
hiddens1=  []; 
hidden  =(torch.zeros(1,1, hidden_dim), torch.zeros(1,1, hidden_dim)) 
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    outputs1.append(out); hiddens1.append(hidden)

In [35]:
len(outputs1) # ths should be equal to sequence (5)
outputs1[0].shape
# You expect all the time sequence hiddens are equal
for i in range(T):
    k = (hiddens1[i][0] == outputs1[i]).sum() 
    assert k ==128

In [36]:
cat_input = torch.cat(inputs).view(len(inputs),1, -1)
# as you can see the dimension are seq_len, batch, input_size
cat_input.shape # this is what you should do in your case as well.

torch.Size([5, 1, 3])

In [37]:
hidden=(torch.zeros(1,1, hidden_dim), torch.zeros(1,1, hidden_dim))
out2, hidden2 = lstm(cat_input, hidden)

In [42]:
out2.shape # seq_len, batch, hidden_dim (5,1,128) contains all hiddens
hidden2[0].shape # 1,1,128 'caz it is only final hidden 
(hidden2[0] == out2[-1,:,:]).sum()

tensor(128)

In [43]:
# this shows us that both cases calculate the same output
(hiddens1[-1][0] == hidden2[0]).sum()

tensor(128)

In [54]:
# which one do you need in your thesis
_, hidden3 = lstm(cat_input, hidden) # you only need final hidden
_, hidden4 = lstm(cat_input)
(hidden3[0] == hidden4[0]).sum()

tensor(128)

In [49]:
# you need to reduce the dimenision into 2
# you need to check linear layer from pytorch
W = torch.randn(hidden_dim, 2)
torch.mm(hidden3[0].view(1,128), W)

tensor([[ 0.3600, -0.1191]], grad_fn=<MmBackward>)